In [8]:
# Specifying all the imports
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool, tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.prompts.prompt import PromptTemplate
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.agents import create_react_agent, AgentExecutor
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from dotenv import load_dotenv
import os

In [9]:
# Loading dotenv and specifying tools
load_dotenv()

@tool
def search_tool(query : str):
    """ 
    Takes a user query and performs an online search to provide
    an answer to the query
    """
    
    search = DuckDuckGoSearchRun()
    answer = search.run(query)
    return answer

@tool
def create_embeddings(query : str):
    """
    Uses the content from the file path to create embeddings on the inputted data. Using the query
    provided, perform a similarity search on the query and return the output. If you can not provide a response,
    say I don't know rather than providing an incorrect response.
    """

    #file_path = r'C:\GenAI\Final Project\GenAIGroupProject\apple.txt'
    file_path = r'./apple.txt'
    file_path = file_path.strip()
    loader = TextLoader(file_path, encoding= 'utf-8')
    document = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunk = text_splitter.split_documents(documents=document)
    embeddings = OpenAIEmbeddings(api_key=os.getenv('OPENAI_API_KEY'))
    vectorstore = FAISS.from_documents(chunk, embeddings)

    docs = vectorstore.similarity_search(query)
    return docs[0].page_content


@tool
def query_database(query : str):
    """
    This function queries the SQL database based on the query that the user provides
    to provide insight into the current client stock portfolio
    """

    llm = ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY'), model='gpt-3.5-turbo', temperature=0.2)
    db = SQLDatabase.from_uri('sqlite:///client_db')
    agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)
    ans = agent_executor.invoke(query)
    return ans

@tool
def portfolio_allocation(query : str):
    """
    Based on the allocation stratergy provided as a query, change the allocation in the
    client stock portfolio in the SQL database
    """
    #query = Reallocte the portfolio for client 1 with a balanced strategy

    #  1. Read in the client database using the query database tool
    s1_query = f"Return the values of the stock holdings for client_id {query} in the sql database"
    query_database(s1_query)
    
    #  2. Determine allocation stratergy based on query
    
    

    #  3. Read in the Financials from the 10k

    #  4. Get current information from online

    #  5. Apply the stratergy we determined on the information from steps 3 and 4!

    #  6. Translate this into a percentage allocation for each company and write to database

 

    

tools = [Tool(name="search_tool", func=search_tool, description="Tool for performing online search operations"),
         Tool(name="create_embeddings", func=create_embeddings, description="Uses the content from the file path to create embeddings on the inputted data. Using the query provided, perform a similarity search on the query and return the output. If you can not provide a response,say I don't know rather than providing an incorrect response."),
        Tool(name='query_database', func=query_database, description= "This function queries the SQL database based on the query that the user provides to provide insight into the current client stock portfolio")]

In [33]:
template = """
You need to use the tools to find the appropriate action to take. 
Only use the query_database and portfolio_allocation tools when a client is mentioned in the query, and if further queries are presented alongside client, then use all tools available.
Otherwise, ignore these two tools and use the embeddings first
to see if you can get relevant responses from the similarity search from the user's query. If you do get relevant
responses, please return the responses from the similarity search. If you can't get relevent results from the embeddings,
use the seatch tool and get the reponse from there. If you still can't find an answer from an online search, please do not make up an
answer and return 'Sorry I can not answer your query' instead.

You have access to the following tools:
{tools}

Question: the input question you must answer

Thought: you should always think about what to do

Action: the action to take, should be one of [{tool_names}]

Action Input: the input to the action

Observation: the result of the action

... (this Thought/Action/Action Input/Observation can repeat N times)

Thought: I now know the final answer

Final Answer: the final answer to the original input question

Begin!

Question: {input}

Thought:{agent_scratchpad}
"""

# Define the query and create the prompt
#query = 'return the values for client_id 1 in the sql database and also tell me the current apple stock price'
#query = 'Tell me the current apple stock price'
#query = 'return me the entire stocks table within the client database as a string'
#query = "The Company continues to develop new technologies to enhance existing products and services and to expand the range of its offerings through research and development RD licensing of intellectual property and acquisition of thirdparty businesses and technology"
#query = "Give me a sentence from the apple 10-k report"
#query = "what is the net sales of Macs in 2021"
#query =  "add a new client to the database with random stock allocations, provide the sql query to do so and execute it"
query =  "set the stock allocations for client_id 1 to 5,5,90. provide the sql query to do so and execute it on the database" #remeber to pull numbers from another query into this framework

prompt = PromptTemplate(input_variables=['input','tools', 'agent_scratchpad', 'tool_names'], template=template)

# Initialize the language model
llm = ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY'), model='gpt-3.5-turbo', temperature=0.2)


# Create the agent using the language model and the toolset
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

# Create the agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

# Execute the agent with the given input
response = agent_executor.invoke({'input': query, 'tools' : tools, 'tool_names' : [tool.name for tool in tools]})

print(response['output'])



> Entering new AgentExecutor chain...
I see that the query involves a specific client_id, so I should use the query_database tool to retrieve and update the stock allocations for the client.

Action: query_database
Action Input: client_id=1, stock_allocations=[5, 5, 90]

> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


stocks
Invoking: `sql_db_schema` with `{'table_names': 'stocks'}`



CREATE TABLE stocks (
	client_id INTEGER, 
	"AAPL" INTEGER, 
	"MSFT" INTEGER, 
	"NVIDIA" INTEGER, 
	PRIMARY KEY (client_id)
)

/*
3 rows from stocks table:
client_id	AAPL	MSFT	NVIDIA
1	30	30	40
2	20	40	40
3	10	10	80
*/
Invoking: `sql_db_query` with `{'query': 'SELECT client_id, "AAPL", "MSFT", "NVIDIA" FROM stocks WHERE client_id = 1'}`
responded: The `stocks` table contains information about stock allocations for different clients. Here is the schema of the `stocks` table:

- client_id: INTEGER (Primary Key)
- "AAPL": INTEGER
- "MSFT": INTEGER
- "NVIDIA": INTEGER